In [1]:
import numpy as np
def DMD(data, r):
    ## Build data matrices
    X1 = data[:,:-1]
    X2 = data[:,1:]
    ## Perform singular value decomposition on X1
    u, s, v = np.linalg.svd(X1, full_matrices = False)
    ## Compute the Koopman matrix
    A_tilde = u[:, : r].conj().T @ X2 @ v[: r, :].conj().T * np.reciprocal(s[: r])
    ## Perform eigenvalue decomposition on A_tilde
    Phi, Q = np.linalg.eig(A_tilde)
    ## Compute the coefficient matrix
    Psi = X2 @ v[: r, :].conj().T @ np.diag(np.reciprocal(s[: r])) @ Q
    A = Psi @ np.diag(Phi) @ np.linalg.pinv(Psi)
    
    return A_tilde, Phi, A

In [2]:
def DMDForecast(data, r, pred_step):
    N, T = data.shape
    _, _, A = DMD(data, r)
    mat = np.append(data, np.zeros((N, pred_step)), axis = 1)
    for s in range(pred_step):
        mat[:, T + s] = (A @ mat[:, T + s - 1]).real
    return mat[:, - pred_step :]

In [3]:
import pandas as pd
data = pd.read_csv('DemandWithFeatures.csv')

In [4]:
data = data.drop(columns = ['Time', 'Day', 'Month', 'Year', 'Winter', 'Spring', 'Summer', 'Fall'])

In [26]:
data = data[:24000]

In [27]:
def minmaxscaler(data):
    minX = data.min()
    maxX = data.max()
    scaled_X = (X-minX)/(maxX-minX)
    return scaled_X, minX, maxX

In [28]:
data.Demand.values[24:]

array([11,  6,  9, ...,  4,  3,  0], dtype=int64)

In [70]:
X = [data.Demand.values[i:int(len(data)/2)+i] for i in range(int(len(data)/2))]

In [83]:
hankel_matrix = np.vstack(X)

In [84]:
np.shape(X)

(12000, 12000)

In [85]:
pred_step = 56
r = 12000

In [ ]:
modes = DMDForecast(np.array(X), r, pred_step)

print(modes)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(modes[0])